In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/ScoreGuardian')

import core.SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

def avance_codmes_col(base, avance_mes):
    for l in range(0, avance_mes + 1, 1):
        base = base.withColumn(f"CODMES_F{l}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), l), "yyyyMM"))
        base = base.withColumn(f"CODMES_F{l}", col(f"CODMES_F{l}").cast("integer"))
    return base

In [0]:
# ltv_malita = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_EXT_SEG_AGRUP_LTV_202504')

# hip_ahol_2 = hip_ahol_2.alias("A") \
#     .join(ltv_malita.alias("F"), F.col("A.CODSOLICITUD") == F.col("F.CODSOLICITUD"), "left") \
#     .join(hip_ahol_2.alias("G"), (F.col("A.CODSOLICITUD") == F.col("G.CODSOLICITUD")) & (F.col("A.CODCLAVEOPECTA") == F.col("G.CODCLAVEOPECTA")), "left") \
#     .select("A.*", F.coalesce(F.col("F.LTV"), F.col("G.LTV_RBM")).alias("LTV_CORREGIDO_0") ,F.col("F.LTV").alias("LTV_EXT"))

In [0]:
# Importando base y configuraciones iniciales
base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hipahlo")
base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0)))
base_seguimiento = base_seguimiento.withColumn("DEF4", when(col("DISTANCIA_DEF24") <= 4, lit(1)).otherwise(lit(0))) 

base_seguimiento.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_seguimiento.groupBy("CODCLAVEOPECTA", "CODMES").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
base_seguimiento = base_seguimiento.withColumn("FLG_VC_VIVIENDA_RBM", F.when(F.col("PRS_ValorComVivienda") <= 355100, 1).otherwise(0))
base_seguimiento = base_seguimiento.withColumn("ACT_HIP", when(col("CODMES")<202308, "ANTES").otherwise(col("ACT_HIP")))

In [0]:
pd_cal='PD_2024Q4vf'#vigente
pd_rbm='PD_RBM'
pd_trc='PD_FINAL_BASE'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Ahorro Local'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Ahorro Local',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Ahorro Local',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento,
  pd1                   = pd_cal,
  pd2                   = pd_rbm,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 AND FLG_VC_VIVIENDA_RBM=1 and ACT_HIP IN ('EMISION MEDIO DE PAGO', 'DESEMBOLSO', 'ANTES')",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

# Segmento No Cliente

## LTV

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('LTV_CREADA', col('MTOAPROBADO_SOLES')/col('CSOL_ValorComVivienda'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('LTV_SALESFORCE', when(col('CODMES')<=202502, col('LTV_SALESFORCE')).otherwise(col('LTV_CREADA')))

In [0]:
# Cortamos la variable
plazo = Obj_Seg.CutVar(variables=['LTV_SALESFORCE', 'LTV_SUITE'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

In [0]:
driver = 'LTV_SALESFORCE_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202105',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08,

)

In [0]:

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
  'LTV_DEFINITIVA_Q',
  when((col('LTV_SUITE') >= 0) & (col('LTV_SUITE') <= 0.40), "0. [0, 40%]")
  .when((col('LTV_SUITE') > 0.4) & (col('LTV_SUITE') <= 0.60), "1. (40%, 60%]")
  .when((col('LTV_SUITE') > 0.6) & (col('LTV_SUITE') <= 0.70), "2. (60%, 70%]")
  .when((col('LTV_SUITE') > 0.7) & (col('LTV_SUITE') <= 0.80), "3. (70%, 80%]")
  .when((col('LTV_SUITE') > 0.8) & (col('LTV_SUITE') <= 0.90), "4. (80%, 90%]")
  .when((col('LTV_SUITE') > 0.9) & (col('LTV_SUITE') <= 1), "5. (90%, 100%]")
)
driver = 'LTV_DEFINITIVA_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202201',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08,
  metrica_secundaria = 'Media'
)

In [0]:
fc = pd.DataFrame([(1, 1), (1, 1), (1, 1)], columns=['CODMES', 'FAC_CAL_DRI_N'])
fm = pd.DataFrame([(1, 1), (1, 1), (1, 1)], columns=['CODMES', 'FAC_CAL_DRI_M'])

In [0]:
caldri_veh_app, rc_dri, rm_dri = Obj_Seg.CalDri(
  driver             = 'LTV_DEFINITIVA_Q',
  query_uso          = f"CODMES >= 202110 AND CODMES <= 202304",
  pd_seg             = 'PD_2024Q4vf',
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0,
  cast_int           = True, 
  proys_def_temp     = 3,
  titulo             = f'LTV_DEFINITIVA_Q',
  dim_grafico        = (26, 6.5),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv = True
)

In [0]:
caldri_veh_app, rc_dri, rm_dri = Obj_Seg.CalDri(
  driver             = 'LTV_SUITE_Q',
  query_uso          = f"CODMES >= 202110 AND CODMES <= 202304",
  pd_seg             = 'PD_2024Q4vf',
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0,
  cast_int           = True, 
  proys_def_temp     = 3,
  titulo             = f'LTV_SUITE_Q',
  dim_grafico        = (26, 6.5),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv = True
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
  query_uso         ='CODMES >= 202201',
  cast_int          =True,
  codmes_inicio1    =202011,
  codmes_fin1       =202204,
  codmes_inicio2    =202501,
  codmes_fin2       =202504,
  dim_grafico       =(25, 6),
  pos_leyenda       =(0.5, -0.25),
 
  # Parametros de figura final
  tipo_vista        = 'M',
  ncolumnas         = 2,
  nfilas            = 1,
  variables         = plazo,
  dim_grafico_total = (21, 6),
  ncol_leyenda      =5,
  tamaño_leyenda    = 8.25,
  vspacio           =-0.45,
  hspacio           =0,
  borde             = 0.08
)

In [0]:
# Corrgiendo LTV para segmento Cliente
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_AJUSTADO_REP", 
    when(col("LTV_SALESFORCE")<0.80017114914425, 0.80017114914425)
    .when(col("LTV_SALESFORCE")>0.8628, 0.8628)
    .otherwise(col("LTV_SALESFORCE"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_AJUSTADO_A2_REP",
    when(col("LTV_AJUSTADO_REP")>=0.8218960830333, col("LTV_AJUSTADO_REP")*( 18.6829109272581)+(-19.0487761651462))
    .when(col("LTV_AJUSTADO_REP")<0.8218960830333,  -3.69336485437273)
    .when(col("LTV_AJUSTADO_REP").isNull(), -3.32089743926051)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_AJUSTADO_A2_REP",
    when(col("LTV_AJUSTADO_A2_REP")<-5.30330490805907,  -5.30330490805907)
    .when(col("LTV_AJUSTADO_A2_REP")>5.30330490805907,  5.30330490805907)
    .otherwise(col("LTV_AJUSTADO_A2_REP"))
)

# Corrigiendo LTV para segmento No Cliente
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_AJUSTADO_REP", 
    when(col("LTV_SALESFORCE")<0.70014975550122, 0.70014975550122)
    .when(col("LTV_SALESFORCE")>0.8628, 0.8628)
    .otherwise(col("LTV_SALESFORCE"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_AJUSTADO_A3_REP",
    when(col("LTV_AJUSTADO_REP")<=0.77197707614449, col("LTV_AJUSTADO_REP")*(13.6280190364747)+(-13.4371609294235))
    .when(col("LTV_AJUSTADO_REP")>0.77197707614449,  -2.91664264000431)
    .when(col("LTV_AJUSTADO_REP").isNull(), -3.05605519073697)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "LTV_AJUSTADO_A3_REP",
    when(col("LTV_AJUSTADO_A3_REP")<-5.30330490805907,  -5.30330490805907)
    .when(col("LTV_AJUSTADO_A3_REP")>5.30330490805907,  5.30330490805907)
    .otherwise(col("LTV_AJUSTADO_A3_REP"))
)

In [0]:
ant_laboral = Obj_Seg.CutVar(variables=['LTV_AJUSTADO_REP'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")
driver = 'LTV_AJUSTADO_REP_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Zona Riesgo

In [0]:
altos = [
    "ATE-LIMA", 
    "CARABAYLLO-LIMA", 
    "EL AGUSTINO-LIMA", 
    "PACHACAMAC-LIMA",
    "PUENTE PIEDRA-LIMA", 
    "SAN JUAN LURIGANCHO",
    "SAN JUAN MIRAFLORES", 
    "VILLA EL SALVADOR", 
    "VILLA MARIA TRIUNFO", 
    "ATE-LIMA", 
    "CARABAYLLO-LIMA",
    "BRE�A-LIMA",
    "BRE#A-LIMA",
    "VENTANILLA-LIMA"
]

oriente = [
    'AMAZONAS',
    'SAN MARTIN',
    'LORETO',
    'MADRE DE DIOS',
    'UCAYALI',
    'HUANUCO',
    'CERRO DE PASCO',
    'JUNIN'
]

norte = [
    'CAJAMARCA',
    'ANCASH',
    'TUMBES',
    'LA LIBERTAD',
    'LAMBAYEQUE',
    'PIURA'
]

sur = [
    'AREQUIPA',
    'AYACUCHO',
    'TACNA',
    'CUZCO',
    'PUNO',
    'APURIMAC',
    'HUANCAVELICA',
    'MOQUEGUA',
    'ICA'
]

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DISTRITO_P", when(col("CODMES")>202307, col("NFT_distritoinmueble")).otherwise(col("ESTADOS_DISTRITO")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEPARTAMENTO_P", when(col("CODMES")>202307, col("NFT_departamentoinmueble")).otherwise(col("ESTADOS_DEPARTAMENTO")))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ZONA_SEG_REP", 
    when((col("DEPARTAMENTO_P") == "CALLAO") & (col("DISTRITO_P") == "VENTANILLA-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "ATE-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "ANCON-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BARRANCO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BELLAVISTA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BRE�A-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BRE#A-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "CARABAYLLO-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LIMA CERCADO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "CHACLACAYO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "CHORRILLOS-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "COMAS-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "EL AGUSTINO-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "INDEPENDENCIA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "JESUS MARIA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LA MOLINA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LA VICTORIA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LINCE-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LOS OLIVOS-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LURIGANCHO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LURIN-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "MAGDALENA DEL MAR-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "MIRAFLORES-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PACHACAMAC-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PUEBLO LIBRE-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PUENTE PIEDRA-LIMA"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "RIMAC-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN BORJA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN ISIDRO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN DE LURIGANCHO"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN LURIGANCHO"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN MIRAFLORES"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN DE MIRAFLORES"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN LUIS'-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN MARTIN DE PORRES"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SANTA ANITA-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SURCO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SURQUILLO-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "VILLA EL SALVADOR"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "VILLA MARIA TRIUNFO"), "ALTO_RIESGO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "MAGDALENA DEL MAR"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SANTIAGO DE SURCO"), "BAJO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN MIGUEL-LIMA"), "BAJO")
    .when((col("DEPARTAMENTO_P").isNull()) | (col("DEPARTAMENTO_P") == "") | (col("DISTRITO_P").isNull()) | (col("DISTRITO_P") == ""), "MISSING")
    .when(col("DEPARTAMENTO_P").isin(oriente), "CENTRO")
    .when(col("DEPARTAMENTO_P").isin(norte), "NORTE")  
    .when(col("DEPARTAMENTO_P").isin(sur), "SUR")
    .otherwise("BAJO")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("FLG_ZONA_RIES_REP", when(col("ZONA_SEG_REP").isin(["ORIENTE", "ALTO_RIESGO", "NORTE"]), 1).otherwise(0))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "FLG_ZONA_RIES_A2_REP",
    when(col("FLG_ZONA_RIES_REP")>=0.31063594313468, col("FLG_ZONA_RIES_REP")*(0.86753195033657)+( -3.79151132823777))
    .when(col("FLG_ZONA_RIES_REP")<0.31063594313468, -3.5220247226455)
    .otherwise(-3.5220247226455)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ZONA_RIES_REP",
    when(col("ZONA_SEG_REP").isin(["ORIENTE", "ALTO_RIESGO"]), "ALTO_RIESGO")
    .when(col("ZONA_SEG_REP").isin(["NORTE", "CENTRO"]), "CENTRO_NORTE")
    .when(col("ZONA_SEG_REP")=="MISSING", "MISSING")
    .otherwise("SUR_LIMA_CALLO")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ZONA_RIES_ST_REP",
    when(col("ZONA_RIES_REP")=="SUR_LIMA_CALLO", -3.377303061)
    .when(col("ZONA_RIES_REP")=="CENTRO_NORTE", -3.059560315)
    .when(col("ZONA_RIES_REP")=="ALTO_RIESGO", -2.661903598)
    .otherwise(-3.056055191)
)

In [0]:
driver = 'ZONA_RIES_REP'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Apalancamiento y Zona

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('APALANCAMIENTO', col('APALANCAMIENTO').cast('float'))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "ZONA_ING_SEG_REP", 
    when((col("DEPARTAMENTO_P") == "CALLAO") & (col("DISTRITO_P") == "VENTANILLA-LIMA"), "CALLAO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "ATE-LIMA"), "LIMA_ESTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BARRANCO-LIMA"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "ANCON-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BELLAVISTA-LIMA"), "CALLAO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BRE�A-LIMA"), "LIMA_CENTRO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BRE#A-LIMA"), "LIMA_CENTRO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "CARABAYLLO-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LIMA CERCADO-LIMA"), "LIMA_CENTRO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "CHACLACAYO-LIMA"), "LIMA_ESTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "CHORRILLOS-LIMA"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "COMAS-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "EL AGUSTINO-LIMA"), "LIMA_ESTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "INDEPENDENCIA-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "JESUS MARIA-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LA MOLINA-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LA VICTORIA-LIMA"), "LIMA_CENTRO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LINCE-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LOS OLIVOS-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LURIGANCHO-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "LURIN-LIMA"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "MAGDALENA DEL MAR-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "MIRAFLORES-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PACHACAMAC-LIMA"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PUEBLO LIBRE-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PUENTE PIEDRA-LIMA"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "RIMAC-LIMA"), "LIMA_CENTRO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN BORJA-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN ISIDRO-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN DE LURIGANCHO"), "LIMA_ESTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN LURIGANCHO"), "LIMA_ESTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN MIRAFLORES"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN JUAN DE MIRAFLORES"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN LUIS-LIMA"), "LIMA_CENTRO")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN MARTIN DE PORRES"), "LIMA_NORTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SANTA ANITA-LIMA"), "LIMA_ESTE")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SURCO-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SURQUILLO-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "VILLA EL SALVADOR"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "VILLA MARIA TRIUNFO"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "MAGDALENA DEL MAR"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SANTIAGO DE SURCO"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN MIGUEL-LIMA"), "LIMA_MODERNA")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "HUANGASCAR"), "LIMA_PROV")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "HUAURA"), "LIMA_PROV")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "HUACHO-LIMA"), "LIMA_PROV")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SANTA MARIA"), "LIMA_PROV")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "PUCUSANA-LIMA"), "LIMA_SUR")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "BARRANCA-LIMA"), "LIMA_PROV")
    .when((col("DEPARTAMENTO_P") == "LIMA") & (col("DISTRITO_P") == "SAN BARTOLO-LIMA"), "LIMA_PROV")
    .when((col("DEPARTAMENTO_P").isNull()) | (col("DEPARTAMENTO_P") == "") | (col("DISTRITO_P").isNull()) | (col("DISTRITO_P") == ""), "MISSING")
    .when(col("DEPARTAMENTO_P").isin("CALLAO"), "LIMA")
    .when(col("DEPARTAMENTO_P").isin(oriente), "CENTRO")
    .when(col("DEPARTAMENTO_P").isin(norte), "NORTE")  
    .when(col("DEPARTAMENTO_P").isin(sur), "SUR")
)

#------------------------------#
#---- APALANCAMIENTO_VF_A3 ----#
#------------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_REP",
    when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='CALLAO'), 4444444444)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='CENTRO'), 5555555555)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_PROV'), 6666666666)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_CENTRO'), 7777777777)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_ESTE'), 1111111111)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_MODERNA'), -1111111111)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_NORTE'), 2222222222)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_SUR'), -2222222222)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='NORTE'), 3333333333)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='ORIENTE'), -3333333333)
    .when(col("ZONA_ING_SEG_REP").isNull(), None)
    .otherwise(col("APALANCAMIENTO"))
)

dummy_list = [4444444444, 4444444444, 6666666666, 7777777777, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333]
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_REP",
    when((~col('APALANCAMIENTO_VF_REP').isin(dummy_list)) & (col("APALANCAMIENTO_VF_REP")<0), 0)
    .when((~col('APALANCAMIENTO_VF_REP').isin(dummy_list)) & (col("APALANCAMIENTO_VF_REP")>14.6738657407407), 14.6738657407407)
    .otherwise(col("APALANCAMIENTO_VF_REP"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_A3_REP",
    when((~col('APALANCAMIENTO_VF_REP').isin(dummy_list)) & (col("APALANCAMIENTO")<=0.57784417966182), col("APALANCAMIENTO_VF_REP")*(2.85727779797169)+( -4.60813537285898))
    .when((~col('APALANCAMIENTO_VF_REP').isin(dummy_list)) & (col("APALANCAMIENTO")>0.57784417966182), -2.95707402762409)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_A3_REP",
    when((col("APALANCAMIENTO_VF_REP").isNull()) | (col("APALANCAMIENTO_VF_REP").isin(5555555555, 6666666666)), -3.30305711397781)
    .when(col("APALANCAMIENTO_VF_REP")==-1111111111,-3.30305711397781)
    .when(col("APALANCAMIENTO_VF_REP")==-2222222222,-3.30305711397781)
    .when(col("APALANCAMIENTO_VF_REP")==-3333333333,-3.30305711397781)
    .when(col("APALANCAMIENTO_VF_REP")==1111111111,-3.30305711397781)
    .when(col("APALANCAMIENTO_VF_REP")==2222222222,-3.82864139648909)
    .when(col("APALANCAMIENTO_VF_REP")==3333333333,-2.69462718077006)
    .when(col("APALANCAMIENTO_VF_REP")==4444444444,-3.30305711397781)
    .when(col("APALANCAMIENTO_VF_REP")==7777777777,-3.19184715248028)
    .otherwise(col("APALANCAMIENTO_VF_A3_REP"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_A3_REP",
    when(col("APALANCAMIENTO_VF_A3_REP")<-5.30330490805907, -5.30330490805907)
    .when(col("APALANCAMIENTO_VF_A3_REP")>5.30330490805907, 5.30330490805907)
    .when(col("APALANCAMIENTO_VF_A3_REP").isNull(), -3.30305711397781)
    .otherwise(col("APALANCAMIENTO_VF_A3_REP"))
)

#------------------------------#
#---- APALANCAMIENTO_VF_A1 ----#
#------------------------------#

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_REP",
    when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='CALLAO'), 4444444444)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='CENTRO'), 5555555555)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_PROV'), 6666666666)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_CENTRO'), 7777777777)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_ESTE'), 1111111111)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_MODERNA'), -1111111111)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_NORTE'), 2222222222)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='LIMA_SUR'), -2222222222)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='NORTE'), 3333333333)
    .when((col("APALANCAMIENTO").isNull()) & (col("ZONA_ING_SEG_REP")=='ORIENTE'), -3333333333)
    .when(col("ZONA_ING_SEG_REP").isNull(), None)
    .otherwise(col("APALANCAMIENTO"))
)

dummy_list = [4444444444, 4444444444, 6666666666, 7777777777, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333]
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_REP",
    when((~col('APALANCAMIENTO_VF_REP').isin(dummy_list)) & (col("APALANCAMIENTO_VF_REP")<0), 0)
    .when((~col('APALANCAMIENTO_VF_REP').isin(dummy_list)) & (col("APALANCAMIENTO_VF_REP")>11.541568), 11.541568)
    .otherwise(col("APALANCAMIENTO_VF_REP"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_A1_REP",
    col("APALANCAMIENTO_VF_REP")*(0.10381308011209)+(-3.88564085924316)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_A1_REP",
    when(col("APALANCAMIENTO_VF_REP").isNull(), -3.3756205398725)
    .when(col("APALANCAMIENTO_VF_REP")==-1111111111,-3.3756205398725)
    .when(col("APALANCAMIENTO_VF_REP")==-2222222222,-3.87120101090789)
    .when(col("APALANCAMIENTO_VF_REP")==-3333333333,-3.3756205398725)
    .when(col("APALANCAMIENTO_VF_REP")==1111111111,-3.3756205398725)
    .when(col("APALANCAMIENTO_VF_REP")==2222222222,-2.39789527279837)
    .when(col("APALANCAMIENTO_VF_REP")==3333333333,-2.38646657697474)
    .when(col("APALANCAMIENTO_VF_REP")==4444444444,-2.00148000021012)
    .when(col("APALANCAMIENTO_VF_REP")==7777777777,-3.45946628978613)
    .when(col("APALANCAMIENTO_VF_REP")==6666666666,-3.3756205398725)
    .when(col("APALANCAMIENTO_VF_REP")==5555555555,-3.3756205398725)
    .otherwise(col("APALANCAMIENTO_VF_A1_REP"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "APALANCAMIENTO_VF_A1_REP",
    when(col("APALANCAMIENTO_VF_A1_REP")<-5.30330490805907, -5.30330490805907)
    .when(col("APALANCAMIENTO_VF_A1_REP")>5.30330490805907, 5.30330490805907)
    .when(col("APALANCAMIENTO_VF_A1_REP").isNull(), -3.3756205398725)
    .otherwise(col("APALANCAMIENTO_VF_A1_REP"))
)

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('APALANCAMIENTO_VF_REP', col('APALANCAMIENTO_VF_REP').cast('float'))
ant_laboral = Obj_Seg.CutVar(variables=['APALANCAMIENTO_VF_REP'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

driver = 'APALANCAMIENTO_VF_REP_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Edad, estado civil y sexo

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_Estcivil_J", 
    when(col("codmes") <= 202307, col("TIPESTCIVIL_ESPEJO")).otherwise(col("SUITE_Estcivil"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "TIPESTCIVIL_N_REP",
    when(
        (col("SUITE_Estcivil_J").isin(['CBS','CON','SEP','VIU','CAS'])) | (col("SUITE_Estcivil_J").isNull()),
        "CAS_OTROS"
    ).otherwise("SOL")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "FLGSEXO_REP",
    when(
        col("FLGSEXO").isNull(),
        "M"
    ).otherwise(col("FLGSEXO"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_EDAD_SEX_EST_REP_1",
    when(col("TIPESTCIVIL_N_REP").isNull() | col("EDAD").isNull() | col("FLGSEXO_REP").isNull(),"MISSING")
    .when((col("TIPESTCIVIL_N_REP") == "CAS_OTROS") & (col("FLGSEXO_REP") == "F"),"NCR1")
    .when((col("TIPESTCIVIL_N_REP") == "CAS_OTROS") & (col("FLGSEXO_REP") == "M"),"NCR2")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") < 26) & (col("FLGSEXO_REP") == "F"),"NCR3")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 26) & (col("EDAD") < 34) & (col("FLGSEXO_REP") == "F"),"NCR4")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 34) & (col("FLGSEXO_REP") == "F"),"NCR5")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") < 26) & (col("FLGSEXO_REP") == "M"),"NCR6")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 26) & (col("EDAD") < 34) & (col("FLGSEXO_REP") == "M"),"NCR7")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 34) & (col("FLGSEXO_REP") == "M"),"NCR8")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_EDAD_SEX_EST_ST_REP_1",
    when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR1", -3.63319054825334)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR2", -3.32568982915401)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR3", -2.45100509811231)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR4", -2.89037175789616)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR5", -3.5481795720108)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR6", -2.30258509299404)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR7", -2.56946403781606)
    .when(col("CAT_EDAD_SEX_EST_REP_1") == "NCR8", -2.89652562347054)
    .otherwise(-3.05605519073697)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_EDAD_SEX_EST_REP_0",
    when(col("TIPESTCIVIL_N_REP").isNull() | col("EDAD").isNull() | col("FLGSEXO_REP").isNull(),"MISSING")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") < 26),"CR1")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 26) & (col("EDAD") < 42) & (col("FLGSEXO_REP") == "M"),"CR2")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 26) & (col("EDAD") < 42) & (col("FLGSEXO_REP") == "F"),"CR3")
    .when((col("TIPESTCIVIL_N_REP") == "SOL") & (col("EDAD") >= 42),"CR4")
    .when((col("TIPESTCIVIL_N_REP") == "CAS_OTROS") & (col("EDAD") < 39),"CR5")
    .when((col("TIPESTCIVIL_N_REP") == "CAS_OTROS") & (col("EDAD") >= 39),"CR6")
    .otherwise("MISSING")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_EDAD_SEX_EST_ST_REP_0",
    when(col("CAT_EDAD_SEX_EST_REP_0") == "CR1", -2.76562005292368)
    .when(col("CAT_EDAD_SEX_EST_REP_0") == "CR2", -3.04154180958528)
    .when(col("CAT_EDAD_SEX_EST_REP_0") == "CR3", -3.56568407082007)
    .when(col("CAT_EDAD_SEX_EST_REP_0") == "CR4", -3.27336401015227)
    .when(col("CAT_EDAD_SEX_EST_REP_0") == "CR5", -3.14168618617707)
    .when(col("CAT_EDAD_SEX_EST_REP_0") == "CR6", -3.78985537145393)
    .otherwise(-3.32089743926051)  # MISSING u otros casos
)

In [0]:
driver = 'CAT_EDAD_SEX_EST_REP_0'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Plazo Aprobado

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEF_CTDPLAZOAPROBADO", when(col("CODMES")>= 202308, col("NFT_PLAZO")).otherwise(col("CTDPLAZOAPROBADO")))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CTDPLAZOAPROBADO_N_REP_0",
    when(col("CTDPLAZOAPROBADO").isNull(), "MISSING")
    .when(col("CTDPLAZOAPROBADO") <= 10, "PLAZO_10")
    .when(col("CTDPLAZOAPROBADO") > 10, "PLAZO_15_20")
    .otherwise("MISSING")
)


Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CTDPLAZOAPROBADO_N_ST_REP_0",
    when(col("CTDPLAZOAPROBADO_N_REP_0") == "PLAZO_10", -3.68171096463532)
    .when(col("CTDPLAZOAPROBADO_N_REP_0") == "PLAZO_15_20", -3.23376443736195)
    .when(col("CTDPLAZOAPROBADO_N_REP_0") == "MISSING", -3.32089743926051)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CTDPLAZOAPROBADO_N_REP_1",
    when(col("CTDPLAZOAPROBADO").isNull(), "MISSING")
    .when(col("CTDPLAZOAPROBADO") <= 10, "PLAZO_10")
    .when((col("CTDPLAZOAPROBADO") > 10) & (col("CTDPLAZOAPROBADO") <= 15), "PLAZO_15")
    .when(col("CTDPLAZOAPROBADO") > 15, "PLAZO_20")
    .otherwise("MISSING")
)


Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CTDPLAZOAPROBADO_N_ST_REP_1",
    when(col("CTDPLAZOAPROBADO_N_REP_1") == "PLAZO_10", -3.5631100668249)
    .when(col("CTDPLAZOAPROBADO_N_REP_1") == "PLAZO_15", -3.02529107579553)
    .when(col("CTDPLAZOAPROBADO_N_REP_1") == "PLAZO_20", -2.84385174226127)
    .when(col("CTDPLAZOAPROBADO_N_REP_1") == "MISSING", -3.05605519073697)
)

In [0]:
driver = 'CTDPLAZOAPROBADO_N_REP_1'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Situación laboral

In [0]:
driver = 'TIPSITUACIONLABORAL_N'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

## Antiguedad Laboral

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SUITE_AntLaboralMeses_J", 
    when(col("codmes") <= 202307, col("ANT_LABORAL_MESES")).otherwise(col("SUITE_AntLaboralMeses"))
)

from pyspark.sql.functions import when, col

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_LAB_ANT_REP",
    when(col("TIPSITUACIONLABORAL_N").isNull() | col("SUITE_AntLaboralMeses_J").isNull(), "MISSING")
    .when((col("TIPSITUACIONLABORAL_N") == "DEP") & (col("SUITE_AntLaboralMeses_J") < 37), "NCA1")
    .when((col("TIPSITUACIONLABORAL_N") == "DEP") & (col("SUITE_AntLaboralMeses_J") >= 37), "NCA2")
    .when(col("TIPSITUACIONLABORAL_N") == "IND", "NCA3")
    .otherwise("MISSING")
)


Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "CAT_LAB_ANT_ST_REP",
    when(col("CAT_LAB_ANT_REP") == "NCA1", -2.68492778368135)
    .when(col("CAT_LAB_ANT_REP") == "NCA2", -3.93360657709532)
    .when(col("CAT_LAB_ANT_REP") == "NCA3", -2.84354045821206)
    .when(col("CAT_LAB_ANT_REP") == "MISSING", -3.05605519073697)
)


In [0]:
ant_laboral = Obj_Seg.CutVar(variables=['SUITE_AntLaboralMeses_J'], num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

driver = 'CAT_LAB_ANT_REP'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
from pyspark.sql.functions import exp
from pyspark.sql.functions import col, log

from pyspark.sql.functions import col, when, exp, round, greatest

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_CEF_FWL",
    -log(1 / col("PD_CEF_FWL") - 1)
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_OPE_CLI_REP",
    + 19.7606 + 
    1.0652*col("APALANCAMIENTO_VF_A3_REP") + 
    0.8225*col("CAT_EDAD_SEX_EST_ST_REP_0") + 
    1.6724*col("CTDPLAZOAPROBADO_N_ST_REP_0") + 
    0.6581*col("FLG_ZONA_RIES_A2_REP") + 
    0.7299*col("LTV_AJUSTADO_A2_REP") + 
    1.1436*col("TIPSITUACIONLABORAL_N_st")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_OPE_CLI_CAL_REP",
    -3.338370259+col("XB_OPE_CLI_REP")*1.007288158
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_OPE_NOCLI_REP",
    + 16.8631 + 
    1.0129*col("APALANCAMIENTO_VF_A1_REP") + 
    0.8328*col("CAT_EDAD_SEX_EST_ST_REP_1") + 
    0.8849*col("CAT_LAB_ANT_ST_REP") + 
    1.5137*col("CTDPLAZOAPROBADO_N_ST_REP_1") + 
    0.6357*col("LTV_AJUSTADO_A3_REP") + 
    0.7218*col("ZONA_RIES_ST_REP")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_OPE_NOCLI_CAL_REP",
    -3.061233813+col("XB_OPE_NOCLI_REP")*0.936436216
)

# Troncal
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_FINAL_BASE_REP",
    when(col("FLAG_CLIENTE")==1, 1.7257+ 0.7581*col("XB_COMP_CLI_CAL")+ 0.7653*col("XB_OPE_CLI_CAL_REP"))
    .when(col("FLAG_CLIENTE")==0, 1.7536 + 0.7694*col("XB_COMP_NOCLI_CAL") + 0.8264 *col("XB_OPE_NOCLI_CAL_REP"))
)

# Troncal ajustado
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_P3_REP",
    + 2.5264843027 + 
    0.4633354446*col("XB_FINAL_BASE_REP") + 
    0.758056468*col("FLG_BANCARIZADO") + 
    0.6013379015*col("FLG_DEPEN_TOT2_st") + 
    0.7977852254*col("SEGMENTO2_st")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_P1_REP",
    + 0.0832037105 + 1.0035121744*col("XB_P3_REP")
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "PD_P1_REP",
    1/(exp(-1*col("XB_P1_REP"))+1)
)

dummy_list = [
    1111111111,
	-1111111111,
	2222222222,
	-2222222222,
	3333333333,
	-3333333333,
	4444444444,
	5555555555,
	6666666666,
	7777777777,
	99999
]

#### PD DISEF
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "PD_DISEF_1",
    when((~col("PD_DISEF").isin(dummy_list)) & (col("PD_DISEF")<0), 0.09580216556787) 
    .when((~col("PD_DISEF").isin(dummy_list)) & (col("PD_DISEF")>0), 0.87092787027359) 
)

from pyspark.sql.functions import log10

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "PD_DISEF_D2",
    when((~col("PD_DISEF").isin(dummy_list)) & (col("PD_DISEF")>=0.30031001399606), log10(col("PD_DISEF")) * 6.25909153062975 + (-1.96152054573695))
    .when((~col("PD_DISEF").isin(dummy_list)) & (col("PD_DISEF")<0.30031001399606), -5.23145889690188)
) 

#### RATIO DE EVOLUTIVO DE MONTO BALANCE
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J",
    when((~col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M").isin(dummy_list)),0.0023248543556)*(exp((col('EXP_PCT_EVOL_SHIP_U3M_RT_U6M')-(-0.12790595 ))/((2.02439643 )-(-0.12790595 ))*(6.38547233898654)))+(-4.35046625457099) 
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J",
    when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M").isNull(), -3.50890807466493)
    .when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M") == -1111111111, -3.87888744383204)
    .when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M") == 1111111111, -3.66356164612964)
    .when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M") == 4444444444, -4.40976338964548)
    .when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M") == 5555555555, -3.87888744383204)
    .when(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M") == 6666666666, -3.89182029811062)
    .otherwise(col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J"))
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('FLAG_DEPENDIENTE', col('flgsituacionlaboraldependiente'))


from pyspark.sql.functions import col, when

# 1. Crear TIPO_EMPLEO_F
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "TIPO_EMPLEO_F",
    when((col("FLAG_DEPENDIENTE") == '1') | (col("FLG_PDH_2_3") == 1), "1. Dependiente")
    .otherwise("Otros")
)

# 2. Crear FLG_DEPENDIENTE
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "FLG_DEPENDIENTE",
    when(col("TIPO_EMPLEO_F") == "1. Dependiente", 1)
    .otherwise(0)
)

# 3. Crear SEGMENTO_MT
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SEGMENTO_MT",
    when((col("FLG_DEPENDIENTE") == 1) & (col("LTV_SALESFORCE") <= 0.8), 1)
    .when((col("FLG_DEPENDIENTE") == 1) & (col("LTV_SALESFORCE") > 0.8), 2)
    .when((col("FLG_DEPENDIENTE") == 0) & (col("LTV_SALESFORCE") <= 0.8), 3)
    .otherwise(4)
)

# Corrigiendo Nuevo calibrado
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_IDIO_2024Q4_REP",
    + 2.9927167711
    + 0.58402489*col("XB_FINAL_BASE_REP")
    + 0.6119697002*col("PD_DISEF_D2")
    + 0.6360502209*col("EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J")
)

# 1. Calcular XB_FWL_2024Q4 según SEGMENTO_MT
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_FWL_2024Q4_REP",
    when(col("SEGMENTO_MT").isin(1, 2),
         1.91243989 +
         0.97182736 * col("XB_IDIO_2024Q4_REP") +
         0.86678477 * col("XB_CEF_FWL"))
    .otherwise(
         0.09995573 +
         0.77288905 * col("XB_IDIO_2024Q4_REP") +
         0.29568435 * col("XB_CEF_FWL"))
)

# 2. Calcular PD_FWL_2024Q4
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "PD_FWL_2024Q4",
    1 / (exp(-col("XB_FWL_2024Q4_REP")) + 1)
)

# 3. Calcular XB_2024Q4 según SEGMENTO_MT
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "XB_2024Q4_REP",
    when(col("SEGMENTO_MT") == 2,
         0.58798239 + 1.03034420 * col("XB_FWL_2024Q4_REP"))
    .when(col("SEGMENTO_MT") == 3,
         0.44258963 + 1.01849005 * col("XB_FWL_2024Q4_REP"))
    .when(col("SEGMENTO_MT") == 4,
         1.30390804 + 1.06821831 * col("XB_FWL_2024Q4_REP"))
    .otherwise(col("XB_FWL_2024Q4_REP"))
)

# 4. Calcular PD_2024Q4
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "PD_2024Q4_REP",
    1 / (exp(-col("XB_2024Q4_REP")) + 1)
)

# 5. Calcular SC_2024Q4
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SC_2024Q4_REP",
    greatest(round(174.25 - 57.708 * col("XB_2024Q4_REP"), 1), lit(3))
)


In [0]:
Obj_Seg.base_monitoreo.persist(StorageLevel.MEMORY_AND_DISK).count()

In [0]:
var_pd = ["XB_FINAL_BASE_REP", "XB_FINAL_BASE"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro      = '',
  codmes_ini        = 202206,
  codmes_fin        = 202312,
  amplitud          = 6,
  disc_target       = [24, 21,18],
  umbral_sup        = 0.4,
  umbral_inf        = 0.3,
  moviles           = True,
  dim_grafico       = (22.5, 6),
  pos_leyenda       = (0.5, -0.35),
  etiquetas         = True,
  gini_abs          = False,
 
  # Parametros de figura final
  ncolumnas         = 2,
  nfilas            = 1,
  variables         = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda      = 5,
  vspacio           = -0.25,
  hspacio           = 4,
  tamaño_etiqueta   = 14 
)

In [0]:
var_pd = ["PD_2024Q4_REP", "PD_2024Q4"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
var_pd = ["XB_OPE_NOCLI_CAL_REP", "XB_OPE_CLI_CAL_REP"]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.4,
  umbral_inf   = 0.3,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
pd_cal='PD_2024Q4'#vigente
pd_rbm='PD_RBM'
pd_trc='PD_FINAL_BASE'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202305
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Ahorro Local'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Ahorro Local',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Ahorro Local',
  mes_seguimiento       = '202504',
  base_monitoreo        = Obj_Seg.base_monitoreo,
  pd1                   = pd_cal,
  pd2                   = 'PD_2024Q4vf',
  pd3                   = pd_rbm,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 AND PD_2024Q4vf IS NOT NULL AND FLG_VC_VIVIENDA_RBM=1",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

In [0]:
Obj_Seg_cal, rc_1, rm_1, fc_1, fm_1 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202106",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', 
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "Best_proy",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(calxmontos['CODMES'].isin([202307, 202306, 202311]), met2_xm + 0.003, met2_xm)",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.12,
  eje_y_xm           = 0.12,
  dim_grafico        = (27,6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   = 10,
  pos_etiquetas_xc   = 10,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rm_1

In [0]:
Obj_Seg.base_monitoreo.createOrReplaceTempView("hip_ahol_2")
spark.sql('''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_app_hipahlo_f
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_app_hipahlo_f'
    AS SELECT * FROM hip_ahol_2
    '''
)